In [1]:
# !pip install beautifulsoup4
# !pip install requests
# !pip install newspaper3k

This notebook scrapes for sports articles from https://www.channelnewsasia.com/sport, which is the sports category in CNA.

## Import packages

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Set link to scrape data from

In [3]:
cna_r = requests.get("https://www.channelnewsasia.com/sport")
coverpage = cna_r.content
soup1 = BeautifulSoup(coverpage, 'html.parser')
cna_r = soup1.find_all('a', href=True)

## Collect top sports news links for the day

In [6]:
links = []

for i in range(len(cna_r)):
    cur_link = cna_r[i]['href']
    if cur_link[0:7] == "/sport/":
        links.append(cur_link)

links

In [7]:
domain = 'https://www.channelnewsasia.com' #domain + link = url to article
url_list = []
content_list = []

news_texts = {} #stores article content in {link : content} format

bad_links = [] #links which could not have their content scraped

In [8]:
from newspaper import Article

for l in links:
    try:
        url = domain + l
        article = Article(url)
        article.download()
        article.parse()
        news_texts[url] = [article.text]
        url_list.append(url)
        content_list.append(article.text)
    except:
        bad_links.append(url)

## Check how many articles were collected

In [10]:
len(news_texts)

21

## Write data to csv

In [11]:
df = pd.DataFrame(list(zip(url_list, content_list)), columns =['url', 'content'])
df.drop_duplicates(inplace=True)

# rename the file
df.to_csv('cna_sports_9mar.csv')